In [2]:
# Simple Google Gemini API Test in Jupyter Notebook

# Step 1: Install required package
!pip install -q google-generativeai python-dotenv

# Step 2: Imports
import google.generativeai as genai
from dotenv import load_dotenv
import os

# Step 3: Load API Key
load_dotenv()  # Load from .env file
# Or set directly:
# GEMINI_API_KEY = "your_api_key_here"  
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

# Step 4: Initialize model
model = genai.GenerativeModel('gemini-1.5-flash')  # Fast and free-tier friendly

# Step 5: Simple nutrition query
def ask_gemini(question):
    """Send a query to Gemini and print the response"""
    try:
        response = model.generate_content(
            f"You are a nutrition expert. Answer concisely: {question}",
            generation_config={
                "max_output_tokens": 500,
                "temperature": 0.7
            }
        )
        print("💬 Response:")
        print(response.text)
    except Exception as e:
        print(f"⚠️ Error: {str(e)}")

# Test with a nutrition question
ask_gemini("What are 3 high-protein vegetarian foods for muscle building?")

# Expected output format:
# 💬 Response:
# 1. Lentils - 18g protein per cooked cup...
# 2. Greek yogurt - 17g protein per 6oz...
# 3. Tofu - 10g protein per ½ cup...


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\WALTON\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


💬 Response:
1. Lentils
2. Tofu
3. Quinoa



In [3]:
ask_gemini("What are 3 high-protein?")

💬 Response:
1. Chicken breast
2. Greek yogurt
3. Lentils



In [ ]:
import streamlit as st
import pandas as pd
import google.generativeai as genai
import os
from dotenv import load_dotenv
from app.data_loader import load_nutrition_data
from app.health_filter import recommend_food
from app.clustering import cluster_foods, get_similar_foods
from app.meal_planner import meal_plan

# 🔑 Google Gemini API Key (from .env file)
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(
    api_key=GEMINI_API_KEY,
    transport='rest'  # Important addition
)
# 📥 Load and process data
df = load_nutrition_data()
df = cluster_foods(df)

# 🔀 Sidebar navigation
page = st.sidebar.selectbox("Select Page", ["Nutrition Recommendation", "Chatbot"])

if page == "Nutrition Recommendation":
    st.title("🍽️ SmartNutriPlan - AI Powered Nutrition App")

    # 🔍 Disease Based Recommendation
    st.header("🔍 Disease Based Recommendation")
    conditions = [
        "obesity", "type 2 diabetes", "high cholesterol (hyperlipidemia)",
        "hypertension (high blood pressure)", "non-alcoholic fatty liver disease (nafld)",
        "coronary artery disease (heart disease)", "stroke", "metabolic syndrome",
        "chronic kidney disease (early stage)", "gastroesophageal reflux disease (gerd)",
        "fatty liver (alcoholic / non-alcoholic)", "gout", "osteoporosis",
        "pcos (polycystic ovary syndrome)", "sleep apnea",
        "fatigue / chronic fatigue syndrome", "depression & anxiety"
    ]
    condition = st.selectbox("Select condition", conditions)
    if st.button("Recommend Foods"):
        recommendations = recommend_food(df, condition)
        st.write(recommendations)

    # 🥗 Meal Planner
    st.header("🥗 Meal Plan")
    target_cal = st.number_input("Target calories", min_value=500, max_value=4000, value=2000, step=50)
    meals = st.number_input("Number of meals", min_value=1, max_value=5, value=3)
    if st.button("Generate Meal Plan"):
        plan = meal_plan(df, target_calories=target_cal, meals=meals)
        st.write(plan)

    # 🤝 Similar Foods
    st.header("🤝 Similar Foods (KMeans Clustering)")
    food_input = st.text_input("Enter food name (example: almond)")
    if st.button("Find Similar Foods"):
        similar = get_similar_foods(df, food_input)
        st.write(similar)

elif page == "Chatbot":
    st.title("🤖 Nutrition Chatbot (Powered by Gemini)")
    user_question = st.text_input("Ask your nutrition question:")

    if st.button("Ask"):
        # Nutrition dataset context
        context = f"""
        Nutrition dataset sample:
        {df.head(10).to_string()}
        
        You are a certified nutrition expert. Provide accurate, science-backed answers about:
        - Food recommendations
        - Meal planning
        - Nutrient information
        - Dietary advice for medical conditions
        
        Answer concisely and cite sources when possible.
        """
        
        
        try:
            # Initialize the Gemini model - ONLY CHANGED THIS LINE
            model = genai.GenerativeModel('gemini-1.5-flash')  # Changed from 'gemini-pro'
            
            # Generate response - ONLY CHANGED THE CONFIG FORMAT
            response = model.generate_content(
                f"{context}\n\nQuestion: {user_question}",
                generation_config=genai.types.GenerationConfig(
                    max_output_tokens=1000,
                    temperature=0.7
                )
            )
            
            # Display the answer
            st.write("💬", response.text)
            
        except Exception as e:
            st.error(f"⚠️ Error getting response: {str(e)}")
            st.info("Note: Gemini has a free tier but may have rate limits")


e:\AI-Powered-Nutrition-App\app\data_loader.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['saturated_fat'].fillna(0, inplace=True)
e:\AI-Powered-Nutrition-App\app\data_loader.py:48: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(valu